In [1]:
#https://stackabuse.com/image-recognition-in-python-with-tensorflow-and-keras/
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [2]:
# Set random seed for purposes of reproducibility
seed = 21

In [3]:
from keras.datasets import cifar10

In [4]:
# loading in the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

In [6]:
# one hot encode outputs
#(one-hot encoding is best used when doing binary classification)
#One-Hotエンコーディング: One-Hot、つまり1つだけ1でそれ以外は0のベクトル（行列）を指し,One-Hotエンコーディングもダミー変数もやっていることはほとんど同じで、カテゴリー変数を0,1の変数に変換して、学習器が学習しやすい形に変換
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [ ]:
""" one-hot-encoding 例
X=np.array([0,1,2,1,0])
X
array([0, 1, 2, 1, 0])
変換後
np.eye(3)[X]
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])
こっちのほうが機械が分かりやすい
"""

In [7]:
#Create the Model
model = Sequential()

In [8]:
#As mentioned, relu is the most common activation, and padding='same' just means we aren't changing the size of the image at all
#  the number of channels/filters we want (that's the 32 below), the size of the filter we want (3 x 3 in this case), the input shape (when creating the first layer)
model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:], padding='same'))
model.add(Activation('relu'))

#or
#model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))

In [9]:
#Now we will make a dropout layer to prevent overfitting, which functions by randomly eliminating some of the connections between the layers (0.2 means it drops 20% of the existing connections)
model.add(Dropout(0.2))

In [10]:
#Batch Normalization normalizes the inputs heading into the next layer, ensuring that the network always creates activations with the same distribution that we desire
model.add(BatchNormalization())

In [11]:
#Now comes another convolutional layer, but the filter size increases so the network can learn more complex representations
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))

In [12]:
#Here's the pooling layer, as discussed before this helps make the image classifier more robust so it can learn relevant patterns. There's also the dropout and batch normalization
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [13]:
"""
That's the basic flow for the first half of a CNN implementation:
Convolutional, activation, dropout, pooling.
You can now see why we have imported 
Dropout, BatchNormalization, Activation, Conv2d, and MaxPooling2d
"""

"\nThat's the basic flow for the first half of a CNN implementation:\nConvolutional, activation, dropout, pooling.\nYou can now see why we have imported \nDropout, BatchNormalization, Activation, Conv2d, and MaxPooling2d\n"

In [14]:
#The exact number of pooling layers you should use will vary depending on the task you are doing, and it's something you'll get a feel for over time. Since the images are so small here already we won't pool more than twice.
#You can now repeat these layers to give your network more representations to work off of
model.add(Conv2D(64, (3, 3), padding='same'))      #Conv2D(16, (3, 3)のコードでは、カーネルという入力データにかける「3×3」の16種類のフィルタを各マスにかけていき、16（枚）の出力データを得られるように指定
model.add(Activation('relu'))       #Rectified Linear Unit: フィルタ後の画像に実施。入力が0以下の時は出力0。入力が0より大きい場合はそのまま出力
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [15]:
#After we are done with the convolutional layers, we need to Flatten the data, which is why we imported the function above. We'll also add a layer of dropout again
model.add(Flatten())     #平坦化（次元削減） – 1次元ベクトルに変換する
model.add(Dropout(0.2))

In [16]:
#Now we make use of the Dense import and create the first densely connected layer. We need to specify the number of neurons in the dense layer. Note that the numbers of neurons in succeeding layers decreases, eventually approaching the same number of neurons as there are classes in the dataset (in this case 10).
#The kernel constraint can regularize the data as it learns, another thing that helps prevent overfitting. This is why we imported maxnorm earlier.
model.add(Dense(256, kernel_constraint=maxnorm(3)))  #全結合層。出力256(投票のようなもの)
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [17]:
#Finally, the softmax activation function selects the neuron with the highest probability as its output, voting that the image belongs to that class
model.add(Dense(class_num))
model.add(Activation('softmax'))

In [18]:
#The optimizer is what will tune the weights in your network to approach the point of lowest loss. The Adam algorithm is one of the most commonly used optimizers because it gives great performance on most problems
epochs = 25
optimizer = 'adam'

In [19]:
#Let's now compile the model with our chosen parameters. Let's also specify a metric to use
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [20]:
# We can print out the model summary to see what the whole model looks like
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0

In [21]:
#Now we get to training the model. To do this, all we have to do is call the fit() function on the model and pass in the chosen parameters.
#Here's where I use the seed I chose, for the purposes of reproducibility
np.random.seed(seed)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=64)

#We'll be training on 50000 samples and validating on 10000 samples.
#Running this piece of code will yield:

Epoch 1/25
782/782 [==============================] - 184s 236ms/step - loss: 1.5236 - accuracy: 0.4589 - val_loss: 1.1552 - val_accuracy: 0.5862
Epoch 2/25
782/782 [==============================] - 184s 235ms/step - loss: 1.0416 - accuracy: 0.6299 - val_loss: 0.9113 - val_accuracy: 0.6765
Epoch 3/25
782/782 [==============================] - 184s 235ms/step - loss: 0.8639 - accuracy: 0.6949 - val_loss: 0.8037 - val_accuracy: 0.7160
Epoch 4/25
782/782 [==============================] - 184s 235ms/step - loss: 0.7616 - accuracy: 0.7337 - val_loss: 0.6797 - val_accuracy: 0.7601
Epoch 5/25
782/782 [==============================] - 183s 234ms/step - loss: 0.7014 - accuracy: 0.7548 - val_loss: 0.6736 - val_accuracy: 0.7652
Epoch 6/25
782/782 [==============================] - 183s 234ms/step - loss: 0.6613 - accuracy: 0.7672 - val_loss: 0.6360 - val_accuracy: 0.7812
Epoch 7/25
782/782 [==============================] - 184s 235ms/step - loss: 0.6311 - accuracy: 0.7799 - val_loss: 0.6338 -

In [22]:
#Now we can evaluate the model and see how it performed. Just call model.evaluate():
# Model evaluation
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.16%
